# Проект Яндекс.Музыка

Задача проекта заключается в анализе данных пользователей Яндекс Музыки и сравнением их поведения.

**Цель исследования** — проверить три гипотезы:

1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Ход исследования**

Данные нужно получить из файла `yandex_music_project.csv`. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится провести предобработку.

Нужно проверить данные на ошибки и оценить их влияние на исследование. Затем,  исправить самые критичные ошибки данных.

Таким образом, исследование пройдёт в три этапа:

 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Обзор данных

Составим первое представление о данных Яндекс Музыки.




In [ ]:
import pandas as pd # импорт библиотеки pandas

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Practicum_project/Проекты_Яндекс_Практикум/Музыка больших городов/yandex_music_project.csv') # чтение файла с данными и сохранение в df

In [ ]:
df.head(10) # получение первых 10 строк таблицы df

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


In [ ]:
df.info() # получение общей информации о данных в таблице df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Итак, в таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

***Описание данных.***

В названиях колонок мы видим, что строчные буквы сочетаются с прописными, а так же видим наличие пробелов.

## Предобработка данных
Исправим обнаруженные недостатки. Затем проверим данные на дубликаты.

### Стиль заголовков


In [ ]:
df.columns # перечень названий столбцов таблицы df

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

***Приведем названия в соответствие с хорошим стилем:***

Для этого переименуем колонки так:
* `'  userID'` → `'user_id'`;
* `'Track'` → `'track'`;
* `'  City  '` → `'city'`;
* `'Day'` → `'day'`.

In [ ]:
df = df.rename(
    columns={
        '  userID': 'user_id',
        'Track': 'track',
        '  City  ': 'city',
        'Day': 'day'
   }
) # переименование столбцов

Проверим результат. Для этого ещё раз выведем на экран названия столбцов:

In [ ]:
df.columns # проверка результатов - перечень названий столбцов

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Пропуски значений

Посчитаем, сколько в таблице пропущенных значений:

In [ ]:
df.isna().sum() # подсчёт пропусков

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Нас интересует активность пользоватей и жанры. Поэтому пропуски в `track` и `artist` для нас не важны и мы заменим их явными обозначениями.

Пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. Поэтому мы так же заменим их явными значениями, и попробуем оценить насколько такое решение будет влиять на наш вывод.

Произведем замену в столбцах `track`, `artist` и `genre` на строку `'unknown'`. Для этого создадим список `columns_to_replace`, переберем его элементы циклом `for` и для каждого столбца выполним замену пропущенных значений:

In [ ]:
columns_to_replace = ['track', 'artist', 'genre'] # перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'
for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')

Убедимся, что в таблице не осталось пропусков. Для этого ещё раз посчитаем пропущенные значения.

In [ ]:
df.isna().sum() # подсчёт пропусков

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### Дубликаты

Посчитаем явные дубликаты в таблице одной командой:

In [ ]:
df.duplicated().sum() # подсчёт явных дубликатов

3826

Удалим явные дубликаты:

In [ ]:
df = df.drop_duplicates() # удаление явных дубликатов

Убедимся в том, что нам удалось избавится от дубликатов

In [ ]:
df.duplicated().sum() # проверка на отсутствие дубликатов

0

Теперь проверим наличие неявных дубликатов в колонке `genre`.

In [ ]:
df_genre = df['genre']# Просмотр уникальных названий жанров
df_genre = df_genre.sort_values()
print(df_genre.unique())

['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'alternativepunk' 'ambient' 'americana' 'animated' 'anime' 'arabesk'
 'arabic' 'arena' 'argentinetango' 'art' 'audiobook' 'author' 'avantgarde'
 'axé' 'baile' 'balkan' 'beats' 'bigroom' 'black' 'bluegrass' 'blues'
 'bollywood' 'bossa' 'brazilian' 'breakbeat' 'breaks' 'broadway'
 'cantautori' 'cantopop' 'canzone' 'caribbean' 'caucasian' 'celtic'
 'chamber' 'chanson' 'children' 'chill' 'chinese' 'choral' 'christian'
 'christmas' 'classical' 'classicmetal' 'club' 'colombian' 'comedy'
 'conjazz' 'contemporary' 'country' 'cuban' 'dance' 'dancehall' 'dancepop'
 'dark' 'death' 'deep' 'deutschrock' 'deutschspr' 'dirty' 'disco' 'dnb'
 'documentary' 'downbeat' 'downtempo' 'drum' 'dub' 'dubstep' 'eastern'
 'easy' 'electronic' 'electropop' 'emo' 'entehno' 'epicmetal' 'estrada'
 'ethnic' 'eurofolk' 'european' 'experimental' 'extrememetal' 'fado'
 'fairytail' 'film' 'fitness' 'flamenco' 'folk' 'folklore' 'folkmetal'
 'folkrock' 

Мы видим следующие неявные дубликаты:
* *hip*,
* *hop*,
* *hip-hop*.

Приведем все к единообразию.

In [ ]:
df = df.replace('hip', 'hiphop') # Устранение неявных дубликатов
df = df.replace('hop', 'hiphop')
df = df.replace('hip-hop', 'hiphop')

Осуществим проверку выполненных действий. Выведем отсортированный список уникальных значений столбца `genre`:

In [ ]:
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

**Выводы**

В разделе предобработки мы обнаружили три проблемы в данных:

- нарушения в стиле заголовков;
- пропущенные значения;
- дубликаты — явные и неявные.

Мы исправили заголовки, чтобы упростить работу с таблицей. А так же прешили проблему дубликатов, теперь исследование станет более точным.

Пропущенные значения заменили на `'unknown'`. Ещё предстоит увидеть, не повредят ли исследованию пропуски в колонке `genre`.

Теперь можно перейти к проверке гипотез.

## Проверка гипотез

### Сравнение поведения пользователей двух городов

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделим пользователей Москвы и Санкт-Петербурга
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.


Оценим активность пользователей в каждом городе. Сгруппируем данные по городу и посчитаем прослушивания в каждой группе.

In [ ]:
df.groupby('city')['genre'].count()


city
Moscow              42741
Saint-Petersburg    18512
Name: genre, dtype: int64

В Москве прослушиваний больше, чем в Петербурге. Это логично, так как самих пользователей в Москве больше.

Теперь сгруппируем данные по дню недели и подсчитаем прослушивания в понедельник, среду и пятницу. Имеем ввиду, что в данных есть информация только о прослушиваниях только за эти дни.


In [ ]:
df.groupby('day')['genre'].count()


В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.


Создадим функцию `number_tracks()`, которая посчитает прослушивания для заданного дня и города. Будем использовать два параметра:
* день недели,
* название города.

In [ ]:
def number_tracks(day, city): # Объявляем функцию с двумя параметрами: day, city.
   # В переменной track_list сохраним те строки таблицы df, для которых
# значение в столбце 'day' равно параметру day и одновременно значение
# в столбце 'city' равно параметру city
    track_list = df[df['day'] == day]
    track_list = track_list[track_list['city'] == city]
# В переменной track_list_count сохраним число значений столбца 'user_id',
# рассчитанное методом count() для таблицы track_list
    track_list_count = track_list['user_id'].count()
# Функция вернет число - значение track_list_count.
    return track_list_count
# Функция для подсчёта прослушиваний для конкретного города и дня.
# С помощью последовательной фильтрации с логической индексацией она
# сначала получит из исходной таблицы строки с нужным днём,
# затем из результата отфильтрует строки с нужным городом,
# методом count() посчитает количество значений в колонке user_id.
# Это количество функция вернёт в качестве результата

Вызовем `number_tracks()` шесть раз, меняя значение параметров — так, чтобы получить данные для каждого города в каждый из трёх дней.

In [ ]:
listening = number_tracks('Monday', 'Moscow') # количество прослушиваний в Москве по понедельникам
print(listening)

15740


In [ ]:
listening = number_tracks('Monday', 'Saint-Petersburg')  # количество прослушиваний в Санкт-Петербурге по понедельникам
print(listening)

5614


In [ ]:
listening = number_tracks('Wednesday', 'Moscow') # количество прослушиваний в Москве по средам
print(listening)

11056


In [ ]:
listening = number_tracks('Wednesday', 'Saint-Petersburg') # количество прослушиваний в Санкт-Петербурге по средам
print(listening)

7003


In [ ]:
listening = number_tracks('Friday', 'Moscow') # количество прослушиваний в Москве по пятницам
print(listening)

15945


In [ ]:
listening = number_tracks('Friday', 'Saint-Petersburg') # количество прослушиваний в Санкт-Петербурге по пятницам
print(listening)

5895


Сведем всю информацию с помощью `pd.DataFrame` в одну таблицу, где
* названия колонок — `['city', 'monday', 'wednesday', 'friday']`;
* данные — результаты, которые мы получили с помощью `number_tracks`.

In [ ]:
data = [['Moscow', 15740, 11056, 15945],['Saint-Petersburg', 5614, 7003, 5895]]
columns = ['city', 'monday', 'wednesday', 'friday']

table = pd.DataFrame(data = data, columns = columns) # Таблица с результатами
print(table)

               city  monday  wednesday  friday
0            Moscow   15740      11056   15945
1  Saint-Petersburg    5614       7003    5895


**Выводы**

Информация указывает на различия в поведении пользователей:

- В Москве наивысшая активность прослушиваний отмечается в понедельник и пятницу, тогда как в среду наблюдается уменьшение интереса.

- В Санкт-Петербурге, наоборот, больше всего музыку слушают по средам. В понедельник и пятницу активность здесь почти равномерно распределена и в целом уступает средней неделе.

Таким образом, полученные данные подтверждают первую гипотезу.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Сохраним таблицы с данными в две переменные:
* по Москве — в `moscow_general`;
* по Санкт-Петербургу — в `spb_general`.

In [ ]:
moscow_general = df[df['city'] == 'Moscow']# получение таблицы moscow_general из тех строк таблицы df,
# для которых значение в столбце 'city' равно 'Moscow'


In [ ]:
spb_general = df[df['city'] == 'Saint-Petersburg']# получение таблицы spb_general из тех строк таблицы df,
# для которых значение в столбце 'city' равно 'Saint-Petersburg'


Создадим функцию `genre_weekday()` которая вернет нам информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [ ]:
# Объявление функции genre_weekday() с параметрами table, day, time1, time2,
# которая возвращает информацию о самых популярных жанрах в указанный день в
# заданное время:
# 1) в переменную genre_df сохраняются те строки переданного датафрейма table, для
#    которых одновременно:
#    - значение в столбце day равно значению аргумента day
#    - значение в столбце time больше значения аргумента time1
#    - значение в столбце time меньше значения аргумента time2
#    Используем последовательную фильтрацию с помощью логической индексации.
# 2) сгруппируем датафрейм genre_df по столбцу genre, возьмем один из его
#    столбцов и посчитаем методом count() количество записей для каждого из
#    присутствующих жанров, получившийся Series запишем в переменную
#    genre_df_grouped
# 3) отсортируем genre_df_grouped по убыванию встречаемости и сохраним
#    в переменную genre_df_sorted
# 4) вернем Series из 10 первых значений genre_df_sorted, это будут топ-10
#    популярных жанров (в указанный день, в заданное время)

def genre_weekday(df, day, time1, time2):
    # последовательная фильтрация
    # оставляем в genre_df только те строки df, у которых день равен day
    genre_df = df[df['day'] == day]
    # оставляем в genre_df только те строки genre_df, у которых время меньше time2
    genre_df = genre_df[genre_df['time'] < time2]
    # оставляем в genre_df только те строки genre_df, у которых время больше time1
    genre_df = genre_df[genre_df['time'] > time1]
    # сгруппируем отфильтрованный датафрейм по столбцу с названиями жанров, возьмём столбец genre и посчитаем кол-во строк для каждого жанра методом count()
    genre_df_grouped = genre_df.groupby('genre')['genre'].count()
        # отсортируем результат по убыванию (чтобы в начале Series оказались самые популярные жанры)
    genre_df_sorted = genre_df_grouped.sort_values(ascending=False)
    # вернём Series с 10 самыми популярными жанрами в указанный отрезок времени заданного дня
    return genre_df_sorted[:10]

Cравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [ ]:
genre_weekday(moscow_general, 'Monday', '07:00', '11:00') # вызовем функцию для утра понедельника в Москве (вместо df — таблица moscow_general)

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [ ]:
genre_weekday(spb_general, 'Monday', '07:00', '11:00') # вызовем функцию для утра понедельника в Петербурге (вместо df — таблица spb_general)

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [ ]:
genre_weekday(moscow_general, 'Friday', '17:00', '23:00') # вызовем функцию для вечера пятницы в Москве

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [ ]:
genre_weekday(spb_general, 'Friday', '17:00', '23:00') # вызовем функцию для вечера пятницы в Петербурге

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Сгруппируем таблицу `moscow_general` по жанру и посчитаем прослушивания треков каждого жанра методом `count()`. Затем отсортируем результат в порядке убывания и сохраним его в таблице `moscow_genres`.

In [ ]:
moscow_genres = (moscow_general.groupby('genre')['genre'].count()).sort_values(ascending=False)# группировка таблицы moscow_general по столбцу 'genre',
# подсчёт числа значений 'genre' в этой группировке методом count(),
# сортировка получившегося Series в порядке убывания и сохранение в moscow_genres

Выведем на экран первые десять строк `moscow_genres`:

In [ ]:
moscow_genres[:10] # просмотр первых 10 строк moscow_genres

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Повторим то же и для Петербурга.

Сгруппируем таблицу `spb_general` по жанру. Посчитаем прослушивания треков каждого жанра. Результат отсортируем в порядке убывания и сохраним в таблице `spb_genres`:


In [ ]:
spb_genres = (spb_general.groupby('genre')['genre'].count()).sort_values(ascending=False)# группировка таблицы spb_general по столбцу 'genre',
# подсчёт числа значений 'genre' в этой группировке методом count(),
# сортировка получившегося Series в порядке убывания и сохранение в spb_genres

Выведем на экран первые десять строк `spb_genres`:

In [ ]:
spb_genres[:10]# просмотр первых 10 строк spb_genres

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге.


## Итоги исследования

Мы проверили три гипотезы и установили:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге.

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.

**Данный проект не содержит исследования проверки статистических гипотез.**
Из данных одного сервиса не всегда можно сделать вывод о всех жителях города.
Проверка статистических гипотез показала бы нам, насколько эти выводы достоверны, исходя из имеющихся данных.
